In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from pyelucidator import Point, BoundingBox, Session
from struct import pack
from scipy import stats
import time

In [ ]:
%matplotlib
plt.rcParams['animation.embed_limit'] = 2**128
plt.rcParams['text.usetex'] = True

In [ ]:
def hits_misses_until(t: float, session: Session) -> tuple[int, int]:
    lower = Point(-1.0, -1.0, -1.0, 0.0)
    upper = Point(1.0, 1.0, 1.0, t)
    bb = BoundingBox(lower, upper)
    states = session.get_metadata("state", bb)
    hits = 0
    misses = 0
    for state in states:
        hits += state["hits"]
        misses += state["misses"]
    return hits, misses

def estimate_pi_at_t(t: float, session: Session) -> float:
    hits, misses = hits_misses_until(t, session)
    if hits + misses == 0:
        return 0
    return hits / (hits + misses) * 4

def calculate_ci_at_t(t: float, ci: float, session: Session) -> float:
    hits, misses = hits_misses_until(t, session)
    z_score = stats.norm.ppf(1 - (1-ci)/2)
    p = hits / (hits + misses)
    se = np.sqrt(p*(1-p)/(hits+misses))
    pi_upper_bound = 4 * (p + z_score * se)
    pi_lower_bound = 4 * (p - z_score * se)
    return pi_upper_bound, pi_lower_bound
    

class MonteCarloSimulation:
    def __init__(self, points_per_step=5):
        self.points_per_step = points_per_step
        self.fig, self.ax = plt.subplots(figsize=(6, 6))
        self.reset()

    def reset(self):
        self.session = Session()
        self.session.add_designation("state", "hits: u64, misses: u64")

        self.ax.clear()
        # Create an array of angles from 0 to 2π
        theta = np.linspace(0, 2 * np.pi, 100)
        
        # Parametric equations for the unit circle
        x = np.cos(theta)
        y = np.sin(theta)
        
        # Plot the unit circle
        self.ax.plot(x, y, color='b')
        self.ax.fill(x, y, color='lightblue', alpha=0.5)  # Fill the circle
        self.ax.axhline(0, color='black', linewidth=0.5, ls='--')
        self.ax.axvline(0, color='black', linewidth=0.5, ls='--')
        self.ax.grid(color='gray', linestyle='--', linewidth=0.5)
        self.ax.set_aspect('equal', adjustable='box')
        self.ax.set_xlim(-1, 1)
        self.ax.set_ylim(-1, 1)

        self.inside, = self.ax.plot([], [], 'rx')
        self.outside, = self.ax.plot([], [], 'bx')
        return self.inside, self.outside

    def run_step(self, t):
        # Generate random points
        points = 2 * np.random.rand(self.points_per_step, 2) - 1
        hits = np.linalg.norm(points, axis=1) < 1
        misses = ~hits

        # Insert data with 'elucidator'
        lower = Point(-1.0, -1.0, -1.0, t)
        upper = Point(1.0, 1.0, 1.0, t + 1.0)
        bb = BoundingBox(lower, upper)
        blob = pack('<QQ', sum(hits), sum(misses))
        self.session.insert_metadata("state", bb, blob)

        return points[hits], points[misses]


    def update(self, frame):
        hits, misses = self.run_step(frame)  # Pass the current frame as time t
        self.inside.set_data(
            np.append(self.inside.get_data(), hits.T, axis=1)
        )
        self.outside.set_data(
            np.append(self.outside.get_data(), misses.T, axis=1)
        )
        return self.inside, self.outside

    def animate(self, num_frames=100):
        ani = animation.FuncAnimation(
            self.fig, self.update, frames=num_frames, init_func=self.reset, blit=True, interval=50
        )
        # Display the animation
        plt.close(self.fig)  # Prevents static display of the figure
        return HTML(ani.to_jshtml())

In [ ]:
FRAMES = 100
POINTS_PER_STEP = 5

In [ ]:
# Create and show the animation
simulation = MonteCarloSimulation(points_per_step=POINTS_PER_STEP)
simulation.animate(num_frames=FRAMES)

In [ ]:
t_vals = np.linspace(1, FRAMES, 100)
ci_95_vals = np.array([calculate_ci_at_t(t, .95, simulation.session) for t in t_vals])
pi_vals = np.array([estimate_pi_at_t(t, simulation.session) for t in t_vals])

In [ ]:
plt.axhline(np.pi, linestyle='dashed', label='$\pi$', color='black', linewidth=1)
plt.plot(t_vals, pi_vals, linestyle='dotted', color='blue', markersize=2, label="$\hat{\pi}$")
plt.plot(t_vals, ci_95_vals[:, 0], linestyle='solid', color='royalblue', markersize=1, label="95\% CI")
plt.plot(t_vals, ci_95_vals[:, 1], linestyle='solid', color='royalblue', markersize=1)

plt.fill_between(t_vals, ci_95_vals[:, 0], ci_95_vals[:, 1], color='lightblue', alpha=0.3)

plt.grid(color='gray', linestyle='--', linewidth=0.2)
plt.ylim(2.5, 3.5)
plt.legend()
plt.show()

In [ ]:
# Final estimate
pi_vals[-1]

In [ ]:
# Final 95% CI values
ci_95_vals[-1]